In [1]:
import csv
from datetime import datetime, timedelta
import os
import pprint

import ee
import pandas as pd

import openet.ptjpl
import openet.ssebop


In [2]:
ee.Initialize(
    project='ee-cmorton',
    opt_url='https://earthengine-highvolume.googleapis.com'
)

In [3]:
et_cell_path = 'et_cell_metadata.csv'
# et_cell_path = os.path.join(os.getcwd(), 'et_cell_metadata.csv')

etf_ws = os.path.join(os.getcwd(), 'etf')
etf_file_prefix = 'etf'

years = list(range(1985, 2024))
overwrite_flag = False


## Read in the list of target grid cells/points to process

In [4]:
et_cell_df = pd.read_csv(et_cell_path)
print(f'ET cell count: {len(et_cell_df)}')


ET cell count: 11


## Extract and save the Landsat scene time series¶

In [5]:
# We will probably not use all the years, but build all the folder
for year in years:
    if not os.path.isdir(os.path.join(etf_ws, f'{year}')):
        os.makedirs(os.path.join(etf_ws, f'{year}'))

etf_files = {
    item for year in years
    for item in os.listdir(os.path.join(etf_ws, f'{year}')) 
    if item.endswith('.csv')
}
print(f'ETf files: {len(etf_files)}\n')
# pprint.pprint(etf_files)


for i, et_cell in et_cell_df.iterrows():
    cell_id = et_cell["GRIDMET_ID"]
    xy = [et_cell['LON'], et_cell['LAT']]
    print(f'{i:>4d} - Cell {et_cell["GRIDMET_ID"]}')
    
    for year in years:
        file_name = f'{etf_file_prefix}_{year}_{cell_id}.csv'
        file_path = os.path.join(etf_ws, f'{year}', file_name)
        if not overwrite_flag and (file_name in etf_files):
            continue
        #print(f'  {year}')

        # Get the data from the collections
        model_obj = openet.ptjpl.Collection(
            collections=['LANDSAT/LC09/C02/T1_L2', 'LANDSAT/LC08/C02/T1_L2', 'LANDSAT/LE07/C02/T1_L2', 'LANDSAT/LT05/C02/T1_L2'],
            model_args={
                'et_reference_source': 'projects/openet/assets/reference_et/conus/gridmet/daily/v1', 
                'et_reference_band': 'eto',
                'et_reference_factor': 1,
                'et_reference_resample': 'nearest',
                # Trying to mimic previous version of PJTPL used for Collection v2.0
                'crop_pm_adjust_flag': True,
                'crop_pm_adjust_source': 'projects/openet/assets/ptjpl/ancillary/alpha/gridmet_1980-2020_dgs',
                'crop_pm_adjust_band': 'pt_eto_ratio',
                'crop_type_source': 'projects/openet/assets/crop_type/v2021a',
                'crop_type_remap': 'CDL',
                'c2_lst_correct': False,
            },
            start_date=f'{year-1}-11-01',
            end_date=f'{year+1}-03-01',
            geometry=ee.Geometry.Point(xy),
            cloud_cover_max=70,
        )
        # model_obj = openet.ssebop.Collection(
        #     collections=['LANDSAT/LC09/C02/T1_L2', 'LANDSAT/LC08/C02/T1_L2', 'LANDSAT/LE07/C02/T1_L2', 'LANDSAT/LT05/C02/T1_L2'],
        #     et_reference_source='projects/openet/assets/reference_et/conus/gridmet/daily/v1', 
        #     et_reference_band='eto',
        #     et_reference_factor=1,
        #     et_reference_resample='nearest',
        #     et_reference_date_type='daily',
        #     start_date=f'{year-1}-11-01',
        #     end_date=f'{year+1}-03-01',
        #     geometry=ee.Geometry.Point(xy),
        #     cloud_cover_max=70,
        #     # filter_args={},
        # )
        scene_info = (
            model_obj
            .overpass(variables=['et_fraction', 'ndvi'])
            .select(['et_fraction', 'ndvi'], ['etf', 'ndvi'])
            .getRegion(geometry=ee.Geometry.Point(xy), scale=30, crs='EPSG:4326')
            .getInfo()
        )
        
        # Reformat the getInfo output so it can be easily read into a dataframe
        scene_dict = {'date': {}, 'etf': {}, 'ndvi': {}, 'landsat': {}}
        column_index = {col_name: i for i, col_name in enumerate(scene_info[0])}
        for row in scene_info[1:]:
            if row[4] is None:
                continue
            date = datetime.utcfromtimestamp(row[3] / 1000.0).strftime('%Y-%m-%d')
            scene_dict['date'][date] = date
            scene_dict['etf'][date] = round(row[column_index['etf']], 4)
            scene_dict['ndvi'][date] = round(row[column_index['ndvi']], 4)
            scene_dict['landsat'][date] = f'Landsat {row[0][-17]}'
        
        # Build and save the scene dataframe
        scene_df = pd.DataFrame.from_dict(scene_dict)
        scene_df['date'] = pd.to_datetime(scene_df['date'], format='%Y-%m-%d')
        scene_df.sort_index(inplace=True)
        scene_df.to_csv(file_path, index=False)

        # break

print('\nDone')


ETf files: 429

   0 - Cell 387098
   1 - Cell 467476
   2 - Cell 473008
   3 - Cell 511799
   4 - Cell 581084
   5 - Cell 605776
   6 - Cell 661167
   7 - Cell 686149
   8 - Cell 688961
   9 - Cell 564161
  10 - Cell 565566

Done
